In [1]:
from pathlib import Path
from tqdm import trange

import torch
import torch.nn.functional as F

import taichi as ti

from differender.volume_raycaster import Raycaster, Mode
from differender.utils import get_rand_pos, in_circles, get_tf

from torchvtk.datasets import TorchDataset
from torchvtk.utils import TFGenerator, tex_from_pts, make_4d
from torchvtk.rendering import show_tf, plot_tf, plot_comp_render_tf

import matplotlib.pyplot as plt

[Taichi] version 0.8.5, llvm 10.0.0, commit 45c6ad48, linux, python 3.8.12
[I 03/17/22 07:48:32.690 877] [shell.py:_shell_pop_print@34] Graphical python shell detected, using wrapped sys.stdout
[TaiGLSL] version 0.0.11


In [2]:
def show_im(im, axs=None):
    if axs is None:
        fig, axs = plt.subplot_mosaic([['rgb', 'alpha', 'wysiwyp'],
                                        ['depth1', 'depth2', 'depth3'],
                                        ['alpha1', 'alpha2', 'alpha3'],
                                        ['sorted1','sorted2','sorted3'],
                                        ['diff1','diff2','diff3']], figsize=(15,25))
    axs['rgb'].imshow(im.squeeze()[:3].permute(1,2,0).cpu().numpy())
    axs['rgb'].set_title('Color')
    axs['rgb'].set_axis_off()
    axs['alpha'].imshow(im.squeeze()[[3,3,3]].permute(1,2,0).cpu().numpy())
    axs['alpha'].set_title('Alpha')
    axs['alpha'].set_axis_off()
    axs['wysiwyp'].imshow(im.squeeze()[[4]].permute(1,2,0).cpu().numpy())
    axs['wysiwyp'].set_title('WYSIWYP')
    axs['wysiwyp'].set_axis_off()    
    
    axs['depth1'].imshow(im.squeeze()[[4]].permute(1,2,0).cpu().numpy())
    axs['depth1'].set_title('Interval 1')
    axs['depth1'].set_axis_off()
    axs['depth2'].imshow(im.squeeze()[[5]].permute(1,2,0).cpu().numpy())
    axs['depth2'].set_title('Interval 2')
    axs['depth2'].set_axis_off()
    axs['depth3'].imshow(im.squeeze()[[6]].permute(1,2,0).cpu().numpy())
    axs['depth3'].set_title('Interval 3')
    axs['depth3'].set_axis_off()
    axs['alpha1'].imshow(im.squeeze()[[7]].permute(1,2,0).cpu().numpy())
    axs['alpha1'].set_title('Alpha at I1')
    axs['alpha1'].set_axis_off()
    axs['alpha2'].imshow(im.squeeze()[[8]].permute(1,2,0).cpu().numpy())
    axs['alpha2'].set_title('Alpha at I2')
    axs['alpha2'].set_axis_off()
    axs['alpha3'].imshow(im.squeeze()[[9]].permute(1,2,0).cpu().numpy())
    axs['alpha3'].set_title('Alpha at I3')
    axs['alpha3'].set_axis_off()

    
    sorted_depth, idx = rgbad[4:7].sort(dim=0)
    axs['sorted1'].imshow(sorted_depth[[0]].permute(1,2,0).cpu().numpy())
    axs['sorted1'].set_title('Sorted Depth 1')
    axs['sorted1'].set_axis_off()
    axs['sorted2'].imshow(sorted_depth[[1]].permute(1,2,0).cpu().numpy())
    axs['sorted2'].set_title('Sorted Depth 2')
    axs['sorted2'].set_axis_off()
    axs['sorted3'].imshow(sorted_depth[[2]].permute(1,2,0).cpu().numpy())
    axs['sorted3'].set_title('Sorted Depth 3')
    axs['sorted3'].set_axis_off()
    
    if 'diff1' in axs:
        diff1 = sorted_depth[1] - sorted_depth[0]
        diff2 = sorted_depth[2] - sorted_depth[1]
        diff3 = sorted_depth[2] - sorted_depth[0]
        axs['diff1'].imshow(diff1[None].permute(1,2,0).cpu().numpy())
        axs['diff1'].set_title('Sorted 2 - 1')
        axs['diff1'].set_axis_off()
        axs['diff2'].imshow(diff2[None].permute(1,2,0).cpu().numpy())
        axs['diff2'].set_title('Sorted 3 - 2')
        axs['diff2'].set_axis_off()
        axs['diff3'].imshow(diff3[None].permute(1,2,0).cpu().numpy())
        axs['diff3'].set_title('Sorted 3 - 1')
        axs['diff3'].set_axis_off()
    
    
def show_hist(im, ax=None, title='Histogram', show_percentages=True):
    tit = f'{title}'
    x = torch.linspace(0.0, 1.0, BINS)
    hst = torch.histc(im, BINS, min=0.0, max=1.0).cpu()
    
    if show_percentages:
        tit = f'{tit}: Transparent: {hst[1:-1].sum()/im.numel():.1%}'
        tit = f'{tit} | Opaque: {hst[-1]/im.numel():.1%}'
        tit = f'{tit} | Visible: {1.0 - hst[0]/im.numel():.1%}'
    if ax is None:
        fig, ax = plt.subplots(1,1, figsize=(5,5))
        fig.title(tit)
    else:
        ax.set_title(tit)
    
    ax.bar(x, hst, width=1/BINS)
    

In [3]:
def show_all_ims(render):
    fig, ax = plt.subplot_mosaic([['full_rgb', 'full_a', 'full_d'],
                                  ['l1_rgb', 'l1_a', 'l1_d'],
                                  ['l2_rgb', 'l2_a', 'l2_d'],
                                  ['l3_rgb', 'l3_a', 'l3_d'],
                                 ], figsize=(15,20))
    ax['full_rgb'].imshow(render[:3].permute(1,2,0))
    ax['full_rgb'].set_title('Full RGB')
    ax['full_rgb'].set_axis_off()
    ax['full_a'].imshow(render[[3]].permute(1,2,0))
    ax['full_a'].set_title('Full Alpha')
    ax['full_a'].set_axis_off()
    ax['full_d'].imshow(render[[4]].permute(1,2,0))
    ax['full_d'].set_title('Full Depth')
    ax['full_d'].set_axis_off()
    
    ax['l1_rgb'].imshow(render[7:10].permute(1,2,0))
    ax['l1_rgb'].set_title('Layer 1 RGB')
    ax['l1_rgb'].set_axis_off()
    ax['l1_a'].imshow(render[[10]].permute(1,2,0))
    ax['l1_a'].set_title('Layer 1 Alpha')
    ax['l1_a'].set_axis_off()
    ax['l1_d'].imshow(render[[4]].permute(1,2,0))
    ax['l1_d'].set_title('Layer 1 Depth')
    ax['l1_d'].set_axis_off()
    
    ax['l2_rgb'].imshow(render[11:14].permute(1,2,0))
    ax['l2_rgb'].set_title('Layer 2 RGB')
    ax['l2_rgb'].set_axis_off()
    ax['l2_a'].imshow(render[[14]].permute(1,2,0))
    ax['l2_a'].set_title('Layer 2 Alpha')
    ax['l2_a'].set_axis_off()
    ax['l2_d'].imshow(render[[5]].permute(1,2,0))
    ax['l2_d'].set_title('Layer 2 Depth')
    ax['l2_d'].set_axis_off()
    
    ax['l3_rgb'].imshow(render[15:18].permute(1,2,0))
    ax['l3_rgb'].set_title('Layer 3 RGB')
    ax['l3_rgb'].set_axis_off()
    ax['l3_a'].imshow(render[[18]].permute(1,2,0))
    ax['l3_a'].set_title('Layer 3 Alpha')
    ax['l3_a'].set_axis_off()
    ax['l3_d'].imshow(render[[6]].permute(1,2,0))
    ax['l3_d'].set_title('Layer 3 Depth')
    ax['l3_d'].set_axis_off()

In [4]:
N_ITEMS = 100000
W, H = 512, 512
TF_RES = 256
BINS = 51
SR = 16.0 # Sampling rate
MODE = Mode.MIDA

out_dir = Path('/mnt/hdd/dome/STDepth_data_multi_mida')
out_dir2 = Path('/mnt/hdd/dome/STDepth_data_multi_wysiwyp')

ds = TorchDataset('/mnt/hdd/shared_datasets/torchvtk/CQ500')
# ds = TorchDataset('/mnt/hdd/shared_datasets/torchvtk/CQ500')
N_VOLUMES = len(ds)

In [5]:
tfgen = TFGenerator(colors='distinguishable', peakgen_kwargs={
    'max_num_peaks': (2,3),
    'height_range': (0.02, 0.4),
    'width_range': (0.005, 0.05),
    'use_hist': False
})

In [ ]:
raycaster = None
skips = 0
visibilities = []
for v in trange(300, N_VOLUMES):
    item = ds[v]
    vol = make_4d(item['vol']).float().to('cuda')
    nam = item['name']
    ti.reset()
    del raycaster
    raycaster = Raycaster(vol.shape[-3:], (W, H), TF_RES, mode=MODE,
        sampling_rate=SR, near=0.1, far=5.0, max_samples=1, ti_kwargs={'device_memory_fraction':0.5})
    for i in trange(N_ITEMS // N_VOLUMES):
        while True:
            lf = get_rand_pos(1).float().to('cuda')
            tf = tex_from_pts(tfgen.generate(), TF_RES).float().to('cuda')
            rgbad = raycaster.raycast_nondiff(vol, tf, lf, sampling_rate=SR, mode=Mode.MIDA).squeeze(0)
            hst = torch.histc(rgbad[3], BINS, min=0.0, max=1.0)
            vis = hst[1:-1].sum()/rgbad[3].numel() # Visibility as in % of pixels with 0.02 < a < 0.9
            if (rgbad[3] > 0.0).float().mean() < 0.1:
                skips += 1
                print(f'Skipping another useless render ({skips})')
                continue
            elif vis < 0.2:
                skips += 1
                print(f'Skipping another too transparent render ({skips})')
                continue
            else:
                rgbad2 = raycaster.raycast_nondiff(vol, tf, lf, sampling_rate=SR, mode=Mode.WYSIWYP).squeeze(0)
                visibilities.append(vis.cpu().float())
                full = rgbad[:4]
                layer1 = torch.cat([rgbad[7:11],  rgbad[[4]]], dim=0)
                layer2 = torch.cat([rgbad[11:15], rgbad[[5]]], dim=0)
                layer3 = torch.cat([rgbad[15:19], rgbad[[6]]], dim=0)
                torch.save({
                    'vol': nam,
                    'tf': tf.squeeze(0).cpu().float(),
                    'look_from': lf.squeeze(0).cpu().float(),
                    'rgba': full.cpu().half(),
                    'layer1': layer1.cpu().half(),
                    'layer2': layer2.cpu().half(),
                    'layer3': layer3.cpu().half(),
                    'visibility': vis.cpu().float()
                }, out_dir/f'{nam}_{i}.pt')
                full = rgbad2[:4]
                layer1 = torch.cat([rgbad2[7:11],  rgbad2[[4]]], dim=0)
                layer2 = torch.cat([rgbad2[11:15], rgbad2[[5]]], dim=0)
                layer3 = torch.cat([rgbad2[15:19], rgbad2[[6]]], dim=0)
                torch.save({
                    'vol': nam,
                    'tf': tf.squeeze(0).cpu().float(),
                    'look_from': lf.squeeze(0).cpu().float(),
                    'rgba': full.cpu().half(),
                    'layer1': layer1.cpu().half(),
                    'layer2': layer2.cpu().half(),
                    'layer3': layer3.cpu().half(),
                    'visibility': vis.cpu().float()
                }, out_dir2/f'{nam}_{i}.pt')
                break
print(f'Average visibility in the dataset: {torch.stack(visibilitieslities).mean():.1%}')

  0%|          | 0/95 [00:00<?, ?it/s]

[Taichi] Starting on arch=cuda



  1%|          | 2/253 [00:16<33:41,  8.05s/it]

Skipping another too transparent render (1)
Skipping another too transparent render (2)



  1%|          | 3/253 [00:27<38:16,  9.19s/it]

...

Skipping another too transparent render (20843)
Skipping another too transparent render (20844)
Skipping another too transparent render (20845)
Skipping another too transparent render (20846)
Skipping another too transparent render (20847)
Skipping another useless render (20848)



 16%|█▌        | 41/253 [07:40<50:00, 14.15s/it]

Skipping another too transparent render (20849)



 17%|█▋        | 42/253 [07:48<44:10, 12.56s/it]

Skipping another too transparent render (20850)
Skipping another too transparent render (20851)


In [ ]:
tf = get_tf('tf1', TF_RES)
lf = in_circles(-1.5)

In [ ]:
show_all_ims(rgbad.cpu())

In [ ]:
plt.imshow(diff.unsqueeze(-1).cpu())

In [ ]:
rgbad1 = raycaster.raycast_nondiff(vol, tf, lf, sampling_rate=SR, mode=Mode.MIDA, layer_dist_thresh=1e-2).squeeze(0)
#rgbad2 = raycaster.raycast_nondiff(vol, tf, lf, sampling_rate=SR, mode=Mode.WYSIWYP).squeeze(0)
print((rgbad[7:10] == rgbad[15:18]).all(), (rgbad[10] == rgbad[18]).all())
show_all_ims(rgbad1.cpu())
#show_all_ims(rgbad2.cpu())

In [ ]:
(raycaster.vr.exit[100,100] - raycaster.vr.entry[100,100]) / raycaster.vr.sample_step_nums[100,100]

In [ ]:
lf = get_rand_pos(1).float().to('cuda')
tf = tex_from_pts(tfgen.generate(), TF_RES).float().to('cuda')

In [ ]:
full = rgbad1[:4]
layer1 = torch.cat([rgbad1[7:11],  rgbad1[[4]]], dim=0)
layer2 = torch.cat([rgbad1[11:15], rgbad1[[5]]], dim=0)
layer3 = torch.cat([rgbad1[15:19], rgbad1[[6]]], dim=0)
stac = torch.stack([layer1, layer2, layer3])

In [ ]:
val, idx = torch.sort(stac[:, 4], dim=0, stable=True)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].imshow(val[0].cpu())
ax[1].imshow(val[1].cpu())
ax[2].imshow(val[2].cpu())
fig.show()

In [ ]:
stac.shape, idx.shape

In [ ]:
idx.shape

In [ ]:
stac[:,0].shape

In [ ]:
sorted_stac = torch.stack([stac[:, i].gather(dim=0, index=idx) for i in range(stac.size(1))], dim=1)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].imshow(sorted_stac[0, [4]].permute(1,2,0).cpu())
ax[1].imshow(sorted_stac[1, [4]].permute(1,2,0).cpu())
ax[2].imshow(sorted_stac[2, [4]].permute(1,2,0).cpu())
fig.show()

In [ ]:
a = torch.rand(2, 3, 3)
b = torch.zeros_like(a)
i = torch.argsort(a, dim=0)
a.shape, i.shape

In [ ]:
a_sort, _ = a.sort(dim=0)

In [ ]:
a, i

In [ ]:
a_sort

In [ ]:
a[0][i[0]]

In [ ]:
red[idx].shape

In [ ]:
sortstac = stac.permute(1,0,2,3)[:, idx]

In [ ]:
stac.shape

In [ ]:
sortstac = stac.permute(1,0,2,3).cpu()[:, idx]

In [ ]:
back1 = (full - layer1[4] * layer1[:4]) / (1.0 - layer1[4])

In [ ]:
plt.imshow(back1[[3]].permute(1,2,0).cpu())

In [ ]:
plt.imshow(layer1[:3].permute(1,2,0).cpu())

In [ ]:
lt = 5e-3
rgbad1 = raycaster.raycast_nondiff(vol[None], tf, lf, sampling_rate=SR, mode=Mode.MIDA, 
        layer_dist_thresh=lt).squeeze(0)
rgbad2 = raycaster.raycast_nondiff(vol[None], tf, lf, sampling_rate=SR, mode=Mode.WYSIWYP,            
        layer_dist_thresh=lt).squeeze(0)

show_all_ims(rgbad1.cpu())
show_all_ims(rgbad2.cpu())

In [ ]:
rgbad = raycaster.raycast_nondiff(vol, tf, lf, sampling_rate=SR, mode=Mode.WYSIWYP).squeeze(0)
show_all_ims(rgbad.cpu())

In [ ]:
tf = tfgen.generate()
while len(tf) == 8: # single peak
    tf = tfgen.generate()
n_peaks = (len(tf)-2)//6
single_peaks = []
for i in range(n_peaks):
    peak = torch.cat([tf[[0]], tf[6*i+1 : 6*(i+1)+1], tf[[-1]]], dim=0)
    single_peaks.append({'pts': peak, 'tex': tex_from_pts(peak, TF_RES)})
    
rgbad_full = raycaster.raycast_nondiff(vol.float().cuda(), tex_from_pts(tf, TF_RES).float().cuda(),
                                       lf.float().cuda(), sampling_rate=SR).squeeze(0).detach().cpu()
#rgbad_full[4:][(rgbad_full[4:] == 0.0)] = 1.0
show_im(rgbad_full)
print((rgbad_full[3] > 0.0).float().mean())

single_peak_renders = []
for p in single_peaks:
    rgbad = raycaster.raycast_nondiff(vol.float().cuda(), p['tex'].float().cuda(), 
                                      lf.float().cuda(), sampling_rate=SR).squeeze(0)
    
    rgbad[4:][(rgbad[4:] == 0.0)] = 1.0
    
    single_peak_renders.append(rgbad.detach().cpu())
    fig, ax = plt.subplot_mosaic([['hist', 'hist', 'tf1'], 
                                  ['hist', 'hist', 'tf2'], 
                                  ['rgb', 'alpha', 'wysiwyp'],
                                  ['depth1', 'depth2', 'depth3'],
                                  ['alpha1', 'alpha2', 'alpha3'],
                                  ['sorted1', 'sorted2', 'sorted3']], figsize=(15,25), gridspec_kw={
        'width_ratios': [1.0, 1.0, 1.0], 'height_ratios': [0.5, 0.5, 1.0, 1.0, 1.0, 1.0]
    })
    show_hist(rgbad[3], ax['hist'], title='Alpha Histogram')
    show_tf(p['pts'], ax=ax['tf1'], title='Current TF Peak')
    show_tf(tf, ax=ax['tf2'], title='Full Transfer Function')
    show_im(rgbad, axs=ax)
    
    fig.show()

for r, p in zip(single_peak_renders, single_peaks):
    abs_diff = torch.abs(r[:4] - rgbad_full[:4])
    abs_ddif = torch.abs(r[[4]] - rgbad_full[[4]])
    mae = abs_diff.mean()
    pct = (abs_diff.max(dim=0).values > 0.1).float().mean().item()
    fig, ax = plt.subplot_mosaic([['single', 'full', 'diff'],
                                  ['sdepth', 'fdepth', 'ddepth'],
                                  ['salpha', 'falpha', 'dalpha'],
                                  ['tf1', 'tf2', 'hist']], figsize=(15,12), gridspec_kw={
        'width_ratios': [1.0, 1.0, 1.0], 'height_ratios': [5.0, 5.0, 5.0, 2.0]
    })
    ax['single'].imshow(r[:3].permute(1,2,0))
    ax['single'].set_title('Partial RGB')
    ax['single'].set_axis_off()
    
    ax['full'].imshow(rgbad_full[:3].permute(1,2,0))
    ax['full'].set_title('Full RGB')
    ax['full'].set_axis_off()
    ax['diff'].imshow(abs_diff[:3].permute(1,2,0))
    ax['diff'].set_title(f'Abs RGB Difference. Pct: {pct:.1%}')
    ax['diff'].set_axis_off()
    
    ax['sdepth'].imshow(r[[4,4,4]].permute(1,2,0))
    ax['sdepth'].set_title('Partial Depth')
    ax['sdepth'].set_axis_off()
    ax['fdepth'].imshow(rgbad_full[[4,4,4]].permute(1,2,0))
    ax['fdepth'].set_title('Full Depth')
    ax['fdepth'].set_axis_off()
    ax['ddepth'].imshow(abs_ddif.permute(1,2,0))
    ax['ddepth'].set_title('Depth Difference')
    ax['ddepth'].set_axis_off()
    show_tf(p['pts'], ax=ax['tf1'], title='Partial TF')
    show_tf(tf, ax=ax['tf2'], title='Full TF')
    show_hist(abs_diff.max(dim=0).values, ax['hist'], title='Difference Histogram', show_percentages=False)
    
    fig.show()
    